## Training a Neural Network on MNIST with Keras

This Project involves 2 main steps:

*   First we will set the data up to be used by our model
*   The next step involves creating and training our model


### Step 1:

#### Imports

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

#### Create Input Pipeline

Load the MNIST dataset with Tensorflow using the following arguments:

*   shuffle_files=True: Shuffle the data to randomize when training the model
*   as_supervised=True: Will return a tuple rather than a dictionary for the model


In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

#### Build Training and Evaluation Pipeline


*   normalize_img() normalizes images: uint8 -> float32
*   The model being used expects tf.float32 so we must normalize our data

In [ ]:
def normalize_img(image, label):
    return tf.cast(image, tf.float32) / 255., label

# normalize training data
ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)

ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

This pipline is similar to the training pipeline built above with a few small differences:


*   No need to call tf.data.Dataset.shuffle
*   Caching is done after batching as batches can be the same between epochs



In [ ]:
# normalize test data
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)

ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

### Step 2:

#### Creating and Training the model

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=12,
    validation_data=ds_test,
)